In [ ]:
'''

用戶菜單功能介紹

    用戶能透過點擊菜單，進行我方希冀之業務功能。
    
流程
    準備菜單的圖面設定檔
    讀取安全設定檔上的參數
    將菜單設定檔傳給Line
    對Line上傳菜單照片
    檢視現有的菜單
    將菜單與用戶做綁定
    將菜單與用戶解除綁定
    刪除菜單

'''

In [1]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "runMenu",
  "chatBarText": "RunMenu",
  "areas": [
    {
      "bounds": {
        "x": 2,
        "y": 3,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "message",
        "text": "[::text:]監控記錄"
      }
    },
    {
      "bounds": {
        "x": 840,
        "y": 3,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "message",
        "text": "[::flex:]賽事報導"
      }
    },
    {
      "bounds": {
        "x": 2,
        "y": 847,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "message",
        "text": "[::text:]休息企"
      }
    },
    {
      "bounds": {
        "x": 840,
        "y": 847,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "uri",
        "uri": "https://image.slidesharecdn.com/chinesehistory-cv-caocao-140730210730-phpapp01/95/-1-638.jpg?cb=1458796316"
      }
    },
    {
      "bounds": {
        "x": 1678,
        "y": 3,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "message",
        "text": "[::text:]我的歌單"
      }
    },
    {
      "bounds": {
        "x": 1678,
        "y": 847,
        "width": 823,
        "height": 836
      },
      "action": {
        "type": "uri",
        "uri": "tel://0227987987"
      }
    }
  ]
}
"""

In [2]:
'''

讀取安全檔案內的字串，以供後續程式碼調用

'''
import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))


0Bdewxw/4Y6bNl384ESt0NFLHDliIEsLMAr/5XexEqinlrJWRyhyYnLXjkZdC1TGb1Xjk9XDdIz4Y43gPLhgZwLPT7M+F722luOe4IuKc3f1CCbOewKZdOxLSrZ+ZOlLQqFH4hnDemLLeUPBDEHtAwdB04t89/1O/w1cDnyilFU=
a94f8d07d5f147bd6733972acabd2f93
U2c320c43564933364fa94c6434ada8bb


In [3]:
'''

將先前準備的菜單設定檔，以Post消息寄發給Line

    設定Line的遠端位置
    設定消息的基本安全憑證
    寄發消息，並取得回應之Id
    

'''

import requests

# 載入設定檔
menuJson=json.loads(menuRawData)

# 上傳到 Line
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'

# 將認證碼 channel_access_token 放到封包的表頭上
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

# 傳給 Line
lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

# 印出結果
print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)


<Response [200]>
{"richMenuId":"richmenu-a8385cd21b3fdcb8c4d976b574640a4a"}


In [4]:
'''

將先前準備的菜單照片，以Post消息寄發給Line
    取得上面設定檔的Id
    設定Line的遠端位置
    設定消息的基本安全憑證
    上傳照片，並取得回傳成果

'''

# 取得功能選單的 id
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定上傳位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 為封包設定認證的公司章 (channel_access_token)
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 讀取圖片
uploadImageFile=open("../mainmenu/menu3.png",'rb')

# 上傳圖片
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

# 觀看結果
print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-a8385cd21b3fdcb8c4d976b574640a4a
https://api.line.me/v2/bot/richmenu/richmenu-a8385cd21b3fdcb8c4d976b574640a4a/content
<Response [200]>
{}


In [5]:
'''

將選單綁定到特定用戶身上
    取出上面得到的菜單Id及用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}

# 取得菜單的 id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

#將 user_id 和 richmenu_id 組合成一個網址
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 為封包設定安全標籤(蓋公司章)
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 將需求傳給 Line
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U2c320c43564933364fa94c6434ada8bb/richmenu/richmenu-a8385cd21b3fdcb8c4d976b574640a4a
<Response [200]>
{}


In [6]:
'''

檢視用戶目前所綁定的菜單
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

#  https://api.line.me/v2/bot/user/{userId}/richmenu

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)


https://api.line.me/v2/bot/user/U2c320c43564933364fa94c6434ada8bb/richmenu
<Response [200]>
{"richMenuId":"richmenu-a8385cd21b3fdcb8c4d976b574640a4a"}


In [ ]:
'''

解除選單與特定用戶的綁定
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)

In [ ]:
'''

檢視帳號內，有哪些選單
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 設定Line的遠端位置
listMenuEndpoint="https://api.line.me/v2/bot/richmenu/list"

# 設定消息的基本安全憑證
listMenuHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}    

# 發送消息告知
lineLisrMenuResponse = requests.get(listMenuEndpoint,headers=listMenuHeader)
print(lineLisrMenuResponse.text)